# A collection of experiments results, notes and considerations
This playbook is a digital lab book to record experiments and keep track of
the progress.

## X-Lane (custom bypass) jitter locality on FPGA SmartNIC 
> tests ran in date: 06/2023 and 07/2023\
> files: fide-eval repository, playbook bsc-fpga/fullts/*.ipynb, data: bsc-fpga/fullts/kbuf-logtype

This benchmark uses a FPGA SmartNIC with FPGA timestamp functionality to
breakdown the communication latency of the TX and RX path. The setup is a
2 nodes connected with a p2p connection, no switch in between.

```
|SOC1|--->|FPGA1|------->|FPGA2|--->|SOC2|
  ^          ^              ^         ^
 ts1        ts2            ts3       ts4
```
Timestamps ts2 and ts3 are taken respectively after and before the PCIe bus. 
ts1 is taken in the software send loop of the LKM. ts4 is an *optimistical
timestamp*, taken as early as possible in a customised net driver (X-Lane
Linux). It's worth noting that there might be additional non -negligible delay
from ts4 to an application. 

No clock is sync'd (SOC-SOC nor SOC-FPGA), so we obtain the jitter through the difference
of successive timestamps. For ts1: `ts1(n+1)-ts1(n), ts1(n+2)-ts1(n+1)...`, same for ts2, 
ts3 and ts4. Differences when comparing the plots allow us to detect whether the jitter occurred
in the TX path, network, or RX path.

### Relevant benchmark settings
- Hardware: Intel BigSpringCanyon SmartNIC, SOC CPU @ 1.5GHz, FPGA clock @ 300MHz.
- Sending interval: 1ms
- isolcpus: yes
- timing interrupts shielded: yes
- no IRQ on isolated cores
- maxpower state
- logging type: kernel buffer
- number of rounds: 20000


![comparison plot](plots/soc_fpga_comparison_20230705_1732.png)
![comparison plot](plots/soc_fpga_comparison_20230726_1425.png)
<!-- ![comparison plot](plots/soc_fpga_comparison_20230727_1732.png) -->
